In [1]:
"""
Master Notebook 
"""

'\nMaster Notebook \n'

In [1]:
"""Imports
"""
import pandas as pd 
from pathlib import Path 
from dotenv import load_dotenv 
import os
import matplotlib 
import matplotlib.pyplot as plt
import hvplot.pandas
import numpy as np
import alpaca_trade_api as tradeapi
import panel as pn
from datetime import datetime
import requests
import seaborn as sns


# from scipy import stats
# from tabulate import tabulate
# import finance_calculator as fc
# import operator 


In [2]:

#Loading in my environments: dotenv an 

load_dotenv("api.env")
APCA_API_BASE_URL = "https://paper-api.alpaca.markets" 


#Setting up my Alpaca API Keys and creating the Alpaca API object: 

apikey = os.getenv("ALPACA_API_KEY")
secret = os.getenv("ALPACA_SECRET_KEY")

api = tradeapi.REST(
    apikey, secret,
    api_version = "v2"
)

#Calling in my securities and setting up a mock portfolio 

assets = ["ARKK","SPY","IWM"]


#Getting my ticker information from Alpaca API 

timeframe = "1Day"
start_date = pd.Timestamp("2018-12-31", tz = "America/New_York").isoformat()
end_date = pd.Timestamp("2023-12-29", tz = "America/New_York").isoformat()

#Getting my ticker information for the funds: ARKK and SPY 

df_assets = api.get_bars(
    assets,
    timeframe,
    start = start_date,
    end = end_date
).df 

# df_assets.head()

#Dropped column head: Symbol and made SPY & ARKK the symbol head 
SPY = df_assets[df_assets['symbol'] == 'SPY'].drop('symbol', axis = 1)
ARKK = df_assets[df_assets['symbol'] == 'ARKK'].drop('symbol', axis = 1)
IWM = df_assets[df_assets['symbol'] == 'IWM'].drop('symbol', axis = 1) 

#Concat the dataframe to show SPY & ARKK as the column head. 
df_assets_concat = pd.concat([SPY, ARKK, IWM], axis = 1, keys = ["SPY","ARKK", "IWM"])
# df_assets_concat

#Setting up a blank dataframe to input my ARKK & SPY 
df_ARKK_SPY = pd.DataFrame() 

#Isolated the SPY and ARKK close dates 
df_ARKK_SPY["SPY"] = df_assets_concat["SPY"]["close"]
df_ARKK_SPY["ARKK"] = df_assets_concat["ARKK"]["close"]
df_ARKK_SPY["IWM"] = df_assets_concat["IWM"]["close"]

#Dropped the time index from the date 
df_ARKK_SPY.index = df_ARKK_SPY.index.date
# df_ARKK_SPY.head()

#Calculated the daily returns of ARKK & SPY 
df_ARKK_SPY_daily = df_ARKK_SPY.pct_change().dropna()
# df_ARKK_SPY_daily.head()

#Plotting the daily returns 
df_ARKK_SPY_daily.hvplot.line(title = "ARKK, SPY, IWM Daily Returns")

######


df_ARKK_SPY.head()

,SPY,ARKK,IWM
2018-12-31,250.08,37.19,133.900
2019-01-02,250.23,37.42,134.695
2019-01-03,244.15,36.19,132.310
2019-01-04,252.39,38.43,137.210
2019-01-07,254.29,40.10,139.630


In [4]:
# arkk_df_calc = df_ARKK_SPY.ARKK
# spy_df_calc = df_ARKK_SPY.SPY

# arkk_list = []
# spy_list = []
# for i in arkk_df_calc:
#     arkk_list.append(i)
# for j in spy_df_calc: 
#     spy_list.append(j)


# beta = fc.get_beta(df_ARKK_SPY.ARKK, df_ARKK_SPY.SPY, risk_free_rate = None, window = 1259, annualiser = 5)
#beta = fc.get_beta(arkk_list, spy_list, risk_free_rate = None, window = 1250, annualiser = 5, tail = True)


In [5]:
# arkk_df_calc.info()







In [6]:
"""Aditya's Section - Data
"""


def get_historical_bitcoin_data(start_date, end_date):
    # Convert start and end dates to Unix timestamp
    start_timestamp = int(datetime.strptime(start_date, "%Y-%m-%d").timestamp())
    end_timestamp = int(datetime.strptime(end_date, "%Y-%m-%d").timestamp())
    # Calculate the number of days between start and end dates
    days_difference = (end_timestamp - start_timestamp) // (24 * 3600) # seconds in a day
    # API endpoint for historical prices
    api_url = 'https://api.coingecko.com/api/v3/coins/bitcoin/market_chart'
    # Define parameters for the API request
    params = {
        'vs_currency': 'usd',
        'from': start_timestamp * 1000, # Multiply by 1000 for milliseconds
        'to': end_timestamp * 1000, # Multiply by 1000 for milliseconds
        'days': days_difference # Add the 'days' parameter
     }
    # Make the API request
    response = requests.get(api_url, params=params)
    
    if response.status_code == 200:
        data = response.json()
        prices = data.get('prices', [])
        return prices
    else:
        print(f"Failed to retrieve Bitcoin data. Status code: {response.status_code}")
        print(f"Error message: {response.text}")
    return None
    
# Set the date range
start_date = '2018-12-31'
end_date = '2023-12-29'
# Get historical Bitcoin data
bitcoin_prices = get_historical_bitcoin_data(start_date, end_date)
# Display the results in a simple table format

dates = [] 
prices_btc = []

if bitcoin_prices:
    # print("{:<20} {:<15}".format("Date", "Price (USD)"))
    for timestamp, price in bitcoin_prices:
        date_str = datetime.fromtimestamp(timestamp / 1000).strftime('%Y-%m-%d')
        if date_str <= end_date:
            dates.append(date_str)
            prices_btc.append(price)
            #print("{:<20} {:<15.2f}".format(date_str, price))

# format = '%Y-%m-%d'
# for i in dates:
#     datetime.datetime.strptime(i, format).date()

# df_bitcoin = pd.DataFrame(prices_btc, dates)

# df_bitcoin.head()

df_bitcoin_raw = pd.DataFrame(
    {"Date":dates,
     "BTC":prices_btc}
)
df_bitcoin_set = df_bitcoin_raw.set_index("Date")
df_bitcoin_set


,BTC
Date,
2019-02-12,3633.965047
2019-02-13,3610.062273
2019-02-14,3589.661830
2019-02-15,3601.229191
2019-02-16,3622.591653
...,...
2023-12-25,43638.236302
2023-12-26,42516.426940
2023-12-27,43418.467775


In [7]:
"""Antonio Section - Data
"""
csvpath = Path('FGRTX.csv')
fgrtx_df = pd.read_csv(csvpath)
# fgrtx_df.head()

#Sorting by "Date" 
# Sort data by `Date` in ascending order
fgrtx_df = fgrtx_df.sort_values("Date", ascending=True)
# fgrtx_df.head()

#Converted into datetime 
fgrtx_df.set_index(pd.to_datetime(fgrtx_df['Date'], infer_datetime_format=True), inplace=True)
# fgrtx_df.head()

#Drop the column
fgrtx_df.drop(columns=['Date','Open','High','Low','Close','Volume'], inplace=True)
fgrtx_df.head()

# Count nulls
fgrtx_df.isnull().sum()
# Drop nulls
fgrtx_df = fgrtx_df.dropna().copy()
fgrtx_df

#Check nulls
fgrtx_df.isnull().sum()

#Check datatypes
fgrtx_df.dtypes



/var/folders/jm/yl4jhp9j0z1b4xdws78jvgjc0000gn/T/ipykernel_5176/2898963672.py:13: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  fgrtx_df.set_index(pd.to_datetime(fgrtx_df['Date'], infer_datetime_format=True), inplace=True)


Adj Close    float64
dtype: object

In [8]:
#Graph of the mutual fund: FGRTX 
fgrtx_df_price_data=fgrtx_df.hvplot(kind="line")
fgrtx_df_price_data

:Curve   [Date]   (Adj Close)

In [9]:
# Calculate the daily return using the 'pct_change()' function
daily_returns_fgrtx = fgrtx_df.pct_change()
daily_returns_fgrtx.head()

# Drop nulls
daily_returns_fgrtx = daily_returns_fgrtx.dropna().copy()
daily_returns_fgrtx

# Rename `Close` Column to be specific to this portfolio.
daily_returns_fgrtx = daily_returns_fgrtx.rename(columns={
    "Adj Close": "FGRTX"
   })

daily_returns_fgrtx.head()


plot_daily_returns_fgrtx=daily_returns_fgrtx.hvplot(figsize=(10,5))
daily_returns_fgrtx.hvplot(kind="line")

:Curve   [Date]   (FGRTX)

In [65]:

# Set File path
csvpath = Path('EFFR.csv')

effr_df = pd.read_csv(csvpath)
effr_df.head()

# Sort data by `Date` in ascending order
effr_df = effr_df.sort_values("Effective Date", ascending=True)
effr_df.head()

# Rename `Close` Column to be specific to this portfolio.
effr_df = effr_df.rename(columns={
    "Effective Date": "Date"
   })

effr_df.head()

#Set Date as Index
effr_df.set_index(pd.to_datetime(effr_df['Date'], infer_datetime_format=True), inplace=True)
effr_df.head()

effr_df_data=effr_df[["Rate (%)"]] #Only includes what's inside the [[]] 
effr_df_data.head()

effr_df_data = effr_df_data.dropna().copy()

#Rename `Close` Column to be specific to this portfolio.
effr_df_data = effr_df_data.rename(columns={
    "Rate (%)": "EFFR % Rate"
   })
effr_df_data.head()

effr_df_data.isnull().sum() 

effr_df_data = effr_df_data.sort_values("Date", ascending=True)
effr_df_data.tail()

effr_df_data_change = effr_df_data.copy()
effr_df_data_change.head()

effr_df_data_change['EFFR % Rate'] = effr_df_data_change['EFFR % Rate']/100
effr_df_data_change

/var/folders/jm/yl4jhp9j0z1b4xdws78jvgjc0000gn/T/ipykernel_5176/2711737802.py:19: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  effr_df.set_index(pd.to_datetime(effr_df['Date'], infer_datetime_format=True), inplace=True)


,EFFR % Rate
Date,
2018-12-31,0.0240
2019-01-02,0.0240
2019-01-03,0.0240
2019-01-04,0.0240
2019-01-07,0.0240
...,...
2023-12-22,0.0533
2023-12-26,0.0533
2023-12-27,0.0533


In [11]:
####Calculating the daily returns:

#FGRTX Mutual Fund Daily Returns
daily_returns_fgrtx 
fgrtx_df
#ARKK & SPY Daily Returns 
daily_returns_arkk_spy = df_ARKK_SPY.pct_change().dropna()
# daily_returns_arkk_spy.head()

#Bitcoin Prices 
daily_returns_bitcoin = df_bitcoin_set.pct_change().dropna()
# daily_returns_bitcoin.head()
##### Joined_daily_returns:


# print((daily_returns_fgrtx),(daily_returns_arkk_spy),(daily_returns_bitcoin))
#Everything prints out well. 

# concat_prices = pd.concat([fgrtx_df,df_ARKK_SPY,df_bitcoin_set], axis = 1, join = 'inner', keys = ["FGRTX","BTC", "SPY","ARKK","IWM"])
# concat_prices



In [45]:
#FGRTX Mutual Fund Daily Returns
daily_returns_fgrtx = daily_returns_fgrtx.reset_index()
daily_returns_fgrtx['Date'] = pd.to_datetime(daily_returns_fgrtx['Date'])
daily_returns_fgrtx = daily_returns_fgrtx.set_index('Date')
# daily_returns_fgrtx = daily_returns_fgrtx.drop(columns=['index'])
daily_returns_fgrtx.index
# daily_returns_fgrtx.index
# fgrtx_df.hea

DatetimeIndex(['2019-01-03', '2019-01-04', '2019-01-07', '2019-01-08',
               '2019-01-09', '2019-01-10', '2019-01-11', '2019-01-14',
               '2019-01-15', '2019-01-16',
               ...
               '2023-12-15', '2023-12-18', '2023-12-19', '2023-12-20',
               '2023-12-21', '2023-12-22', '2023-12-26', '2023-12-27',
               '2023-12-28', '2023-12-29'],
              dtype='datetime64[ns]', name='Date', length=1257, freq=None)

In [14]:
daily_returns_arkk_spy = df_ARKK_SPY.pct_change().dropna()
# daily_returns_arkk_spy.head()
# daily_returns_arkk_spy.index

In [15]:
#Bitcoin Prices 
daily_returns_bitcoin = df_bitcoin_set.pct_change().dropna()
# daily_returns_bitcoin.head()
# daily_returns_bitcoin.index

In [43]:
#Set arkk_spy into date: 
daily_returns_arkk_spy_set = daily_returns_arkk_spy.reset_index()
daily_returns_arkk_spy_set['index'] = pd.to_datetime(daily_returns_arkk_spy_set["index"])
daily_returns_arkk_spy_set['Date'] = daily_returns_arkk_spy_set['index']

daily_returns_arkk_spy_set = daily_returns_arkk_spy_set.set_index('Date') 
# daily_returns_arkk_spy_set_dropped = 
daily_returns_arkk_spy_set.drop(columns=["index"], inplace=True)
daily_returns_arkk_spy_set.index

DatetimeIndex(['2019-01-02', '2019-01-03', '2019-01-04', '2019-01-07',
               '2019-01-08', '2019-01-09', '2019-01-10', '2019-01-11',
               '2019-01-14', '2019-01-15',
               ...
               '2023-12-15', '2023-12-18', '2023-12-19', '2023-12-20',
               '2023-12-21', '2023-12-22', '2023-12-26', '2023-12-27',
               '2023-12-28', '2023-12-29'],
              dtype='datetime64[ns]', name='Date', length=1258, freq=None)

In [42]:
#Set bitcoin index: 
daily_returns_bitcoin_set = daily_returns_bitcoin.reset_index()
daily_returns_bitcoin_set["Date"] = pd.to_datetime(daily_returns_bitcoin_set['Date'])

daily_returns_bitcoin_set = daily_returns_bitcoin_set.set_index('Date')
# daily_returns_bitcoin_set.tail()
daily_returns_bitcoin_set.index

DatetimeIndex(['2019-02-13', '2019-02-14', '2019-02-15', '2019-02-16',
               '2019-02-17', '2019-02-18', '2019-02-19', '2019-02-20',
               '2019-02-21', '2019-02-22',
               ...
               '2023-12-20', '2023-12-21', '2023-12-22', '2023-12-23',
               '2023-12-24', '2023-12-25', '2023-12-26', '2023-12-27',
               '2023-12-28', '2023-12-29'],
              dtype='datetime64[ns]', name='Date', length=1781, freq=None)

In [49]:
#Concating the dataframes 
# daily_returns_fgrtx.index
# daily_returns_arkk_spy_set.index

df_daily_returns_all_assets = pd.concat([daily_returns_fgrtx, daily_returns_arkk_spy_set, daily_returns_bitcoin_set], axis=1, join='inner')

df_daily_returns_all_assets

,FGRTX,SPY,ARKK,IWM,BTC
Date,,,,,
2019-02-13,0.004746,0.003247,0.006554,0.003007,-0.006578
2019-02-14,-0.002699,-0.001782,0.002694,0.002281,-0.005651
2019-02-15,0.012855,0.010639,0.011867,0.015738,0.003222
2019-02-19,0.002004,0.001803,0.000664,0.002945,0.007330
2019-02-20,0.000667,0.001979,0.008182,0.004724,0.015785
...,...,...,...,...,...
2023-12-22,0.000958,0.002010,0.013382,0.009318,0.003512
2023-12-26,0.003830,0.004223,0.014714,0.013004,-0.025707
2023-12-27,0.000477,0.001808,0.008738,0.003528,0.021216


In [67]:
df_daily_returns_all_assets_fed_rate = pd.concat([df_daily_returns_all_assets, effr_df_data_change], axis = 1, join = 'inner')
df_daily_returns_all_assets_fed_rate.hvplot.line()

:NdOverlay   [Variable]
   :Curve   [Date]   (value)

In [ ]:
#Plotting using seaborn 
